In [15]:
import torch 
from unet import UNet
from medical_dataset import MedicalDataset
import torch.optim as optim
from loss import Active_Contour_Loss

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [19]:
label = 1
D_train = MedicalDataset(label, "data_train")
D_test = MedicalDataset(label, "data_test")
D_val = MedicalDataset(label, "data_validation")

100%|██████████| 30/30 [00:00<00:00, 49.35it/s]


In [8]:
train_loader = torch.utils.data.DataLoader(
    D_train, batch_size=32, shuffle=True
)

val_loader = torch.utils.data.DataLoader(
    D_test, batch_size=1, shuffle=False
)

In [11]:
model = UNet(1,1)
model.to(device)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [16]:
optimizer = optim.Adam(
    model.parameters(),
    lr=10**-4
)

In [12]:
def train(model, optimizer, epoch, loader):
    model.train()
    losses=[]

    for batch_idx, (data, target) in enumerate(loader):
        if device=="cuda":
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = Active_Contour_Loss(output, target)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        print(
            "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch,
                batch_idx * len(data),
                len(loader.dataset),
                100.0 * batch_idx / len(loader),
                loss.data.item(),
            ),
            flush=True,
        )
    return losses

In [18]:
train(model, optimizer, 0, train_loader)

KeyboardInterrupt: 